# TPCA / FPCA Demonstration (Minimal)

This notebook provides a **small, illustrative** example of how to measure:

- **TPCA** — Tokens Per Correct Answer
- **FPCA (proxy)** — Latency (ms) Per Correct Answer

under Robbie’s Razor evaluation logic.

**Scope note:** This is a demonstration only (not a benchmark leaderboard). The authoritative evaluation surfaces are the unit tests and benchmark scripts in the repository.

## Setup

We add the repo root to `sys.path` so imports work when running from the `notebooks/` directory,
then import the canonical `RazorMemoryBank` reference implementation.

In [ ]:
import os, sys, time

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from src.razor.memory_bank import RazorMemoryBank

print("Repo root:", repo_root)
print("Imported:", RazorMemoryBank)


## Definitions

We compute TPCA and an FPCA-like proxy using latency (ms) instead of FLOPs.

- **TPCA** = total output tokens / number of correct answers
- **FPCA (latency proxy)** = total latency ms / number of correct answers

Token counting here uses a deterministic proxy: ~4 characters per token.
This avoids external dependencies while still providing a stable ratio.

In [ ]:
def token_proxy(text: str) -> int:
    """Deterministic proxy: tokens ~= max(1, ceil(len(text)/4))."""
    text = text or ""
    return max(1, (len(text) + 3) // 4)


def tpca(num_correct: int, total_tokens: int):
    return None if num_correct == 0 else total_tokens / num_correct


def fpca_latency(num_correct: int, total_ms: float):
    return None if num_correct == 0 else total_ms / num_correct

## Scenario

We simulate a workload with repeated queries. The memory bank short-circuits repeated work.

### Baseline
- Always "infer" (we simulate inference latency)
- Always output a fixed answer length

### Razor (Memory Gate)
- Check memory first
- If hit (high-confidence), return immediately (near-zero latency / tokens)
- Otherwise "infer" once, then store result

In [ ]:
# Synthetic workload (repeats)
queries = [
    "What is 17 * 23?",
    "What is 17 * 23?",
    "What is the capital of France?",
    "What is 17 * 23?",
    "What is the capital of France?",
    "What is 2 + 2?",
    "What is 2 + 2?",
]

# Ground truth mapping (simple correctness)
truth = {
    "What is 17 * 23?": "391",
    "What is the capital of France?": "Paris",
    "What is 2 + 2?": "4",
}

# Simulated inference output
def simulated_model_output(q: str) -> str:
    # Return a short correct answer (make verbose to see TPCA worsen)
    return truth[q]

# Simulated inference latency (ms)
INFER_MS = 250

print("Workload size:", len(queries))
print("Unique queries:", len(set(queries)))

## Run: Baseline (always infer)

In [ ]:
baseline_total_tokens = 0
baseline_total_ms = 0
baseline_correct = 0

for q in queries:
    baseline_total_ms += INFER_MS
    out = simulated_model_output(q)
    baseline_total_tokens += token_proxy(out)
    if out == truth[q]:
        baseline_correct += 1

baseline_tpca = tpca(baseline_correct, baseline_total_tokens)
baseline_fpca = fpca_latency(baseline_correct, baseline_total_ms)

print("Baseline correct:", baseline_correct)
print("Baseline total tokens:", baseline_total_tokens)
print("Baseline total ms:", baseline_total_ms)
print("Baseline TPCA:", baseline_tpca)
print("Baseline FPCA (ms/correct):", baseline_fpca)

## Run: Razor Memory Gate (R4-style short-circuit)

Memory hits incur near-zero incremental cost; misses incur inference cost once then store.

In [ ]:
bank = RazorMemoryBank(capacity=1000, stability_threshold=0.95)

razor_total_tokens = 0
razor_total_ms = 0
razor_correct = 0
razor_hits = 0
razor_misses = 0

for q in queries:
    cached, conf = bank.retrieve(q)
    if cached is not None and conf >= 0.95:
        razor_hits += 1
        out = cached
    else:
        razor_misses += 1
        razor_total_ms += INFER_MS
        out = simulated_model_output(q)
        razor_total_tokens += token_proxy(out)
        # Store only if confidence passes the bank's stability threshold
        bank.store(q, out, confidence=0.99)

    if out == truth[q]:
        razor_correct += 1

hit_rate = razor_hits / len(queries)
razor_tpca = tpca(razor_correct, razor_total_tokens)
razor_fpca = fpca_latency(razor_correct, razor_total_ms)

print("Razor correct:", razor_correct)
print("Razor hits:", razor_hits)
print("Razor misses:", razor_misses)
print("Hit rate:", f"{hit_rate:.2%}")
print("Razor total tokens (misses only):", razor_total_tokens)
print("Razor total ms (misses only):", razor_total_ms)
print("Razor TPCA:", razor_tpca)
print("Razor FPCA (ms/correct):", razor_fpca)

try:
    print("Memory stats:", bank.get_stats())
except Exception:
    pass

## Interpretation

This toy example shows the basic mechanism behind efficiency:

- If a workload contains repetition, **memory hits** can skip expensive inference.
- That reduces both token-related costs (TPCA) and compute/latency proxies (FPCA).

In real systems, savings depend on:
- repetition rate
- verification strategy
- how memory keys are canonicalized
- where the gate sits (before inference)

For formal evaluation, use the benchmark scripts and unit tests in this repository.